In [1]:
# import dependencies
import os
import io
import bs4
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
# from selenium import webdriver
# from lxml import html
import csv
from datetime import datetime
from pytz import timezone
import pandas as pd
import FoodTools

In [2]:
# url of ucsc college 9 & 10 dining hall
url = 'https://nutrition.sa.ucsc.edu/nutframe.asp?sName=UC+Santa+Cruz+Dining&locationNum=40&locationName=Colleges+Nine+%26+Ten+Dining+Hall&naFlag=1'

In [3]:
# scrap the menu data from the webpage
# since the menu data is cannot be scrapped from the html file directly
# we have to make additional requests to get the frame page contents 
with requests.Session() as session:
    response = session.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    for frame in soup.select("frameset frame"):
        frame_url = urljoin(url, frame["src"])
        response = session.get(frame_url)
        frame_soup = BeautifulSoup(response.content, 'html.parser') 
        # print(frame_soup.prettify())

In [4]:
# extract both the meal name and food name from html
meal_name = frame_soup.find_all('div', attrs={'class': 'menusampmeals'})
food_name = frame_soup.find_all('div', attrs={'class': 'menusamprecipes'})

In [5]:
# convert the name of the meal type in html form to a list form
meal_type = [item.string for item in meal_name]
print(meal_type)
print('Length: ', len(meal_type))

['Breakfast', 'Lunch', 'Dinner', 'Late Night']
Length:  4


In [6]:
# convert the food name in html form to a list form
food_list = [item.string for item in food_name]
print(food_list)
print('Length: ', len(food_list))

['Belgian Waffles', "Bob's Pancakes", 'Cage Free Scrambled Eggs', 'Cajun Roasted Red Potatoes', 'Chicken Bacon', 'Hard-boiled Cage Free Eggs', 'Oatmeal Gluten-Free', 'Soyrizo Tofu Scramble', 'Banana Fritters', 'Banana Walnut Muffin', 'Chocolate Muffin', 'Vegan Peanut Granola', 'Bell Peppers', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cheddar Cheese', 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Jalapeno', 'Onions', 'Oven Roasted Turkey', 'Sliced Olived', 'Smoked Ham', 'N.E.Clam Chowder', 'Tomato Bisque Soup', 'Island BBQ Chicken', 'Oven Roasted Chicken Thigh', 'Bombay Chicken Pizza', 'Cheese Pizza', 'Steamed Basmati Rice', 'Masala Vegetables', 'Steamed Seasonal Vegetables', 'Chocolate Buttercream Brownie', 'BAR Wings', 'BBQ Wings', 'Condiments', 'Hawaiian Coleslaw', "Hot 'N Spicy Wings", 'Teriyaki Glaze Wings', 'Vegan Baked Beans', 'Vegan Tenders', 'N.E.Clam Chowder', 'Tomato Bisque Soup', 'Harissa Roasted Chicken', 'Oven Roasted Chicken Thigh', 'Spam Fried Rice', 'Steam

In [7]:
# In case of dupliate food name, the list should be converted to a set
# sort
food_list = sorted(list(set(food_list)))
print(food_list)
print('Sorted length: ', len(food_list))

['Apple and Peach Galette', 'Argentinian Roasted Chicken', 'Artichoke Pesto Turkey Burger', 'BAR Wings', 'BBQ Wings', 'Banana Fritters', 'Banana Walnut Muffin', 'Belgian Waffles', 'Bell Peppers', 'Black Beans', "Bob's Pancakes", 'Bombay Chicken Pizza', 'Braised Kale with Crispy Shallots', 'Cage Free Eggs', 'Cage Free Omelette Bar', 'Cage Free Scrambled Eggs', 'Cajun Roasted Red Potatoes', 'Caribbean Yellow Rice', 'Carribbean Bar', 'Cheddar Cheese', 'Cheese Manicotti with Marinara', 'Cheese Pizza', 'Cheese and Garlic Breadsticks', 'Chicken Bacon', 'Chocolate Buttercream Brownie', 'Chocolate Muffin', 'Condiments', 'Crispy Okra', "Cuban Banana BBQ'd Chicken", 'Curly Fries', 'Feta Cheese', 'Fresh Organic Spinach', 'Fresh Tomato', 'Fried Plantains', 'Hard-boiled Cage Free Eggs', 'Harissa Roasted Chicken', 'Hawaiian Coleslaw', "Hot 'N Spicy Wings", 'Island BBQ Chicken', 'Jalapeno', 'Krinkle Cut Fries', 'Masala Vegetables', 'N.E.Clam Chowder', 'Oatmeal Gluten-Free', 'Onions', 'Oven Roasted Ch

In [9]:
# preference list, labeled by myself
# pref_1 = [1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0 ]
# pref_2 = [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0]
# pref_3 = [0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1]
# pref_4 = [0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
# pref_5 = [1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0]
pref = []
for i in food_list:
    print(i)
    preference = int(input())
    if preference != 0 and preference != 1:
        print('Incrorrect input')
        break
    pref.append(preference)
print(pref)
print('Preference length: ', len(pref))

Apple and Peach Galette
0
Argentinian Roasted Chicken
0
Artichoke Pesto Turkey Burger
0
BAR Wings
0
BBQ Wings
0
Banana Fritters
0
Banana Walnut Muffin
0
Belgian Waffles
0
Bell Peppers
0
Black Beans
0
Bob's Pancakes
0
Bombay Chicken Pizza
1
Braised Kale with Crispy Shallots
0
Cage Free Eggs
0
Cage Free Omelette Bar
0
Cage Free Scrambled Eggs
1
Cajun Roasted Red Potatoes
0
Caribbean Yellow Rice
0
Carribbean Bar
0
Cheddar Cheese
0
Cheese Manicotti with Marinara
0
Cheese Pizza
1
Cheese and Garlic Breadsticks
0
Chicken Bacon
1
Chocolate Buttercream Brownie
0
Chocolate Muffin
0
Condiments
0
Crispy Okra
0
Cuban Banana BBQ'd Chicken
0
Curly Fries
1
Feta Cheese
0
Fresh Organic Spinach
0
Fresh Tomato
0
Fried Plantains
0
Hard-boiled Cage Free Eggs
0
Harissa Roasted Chicken
0
Hawaiian Coleslaw
0
Hot 'N Spicy Wings
0
Island BBQ Chicken
1
Jalapeno
0
Krinkle Cut Fries
1
Masala Vegetables
0
N.E.Clam Chowder
0
Oatmeal Gluten-Free
0
Onions
0
Oven Roasted Chicken Thigh
0
Oven Roasted Turkey
0
Peruvian Ba

In [10]:
# timezone setting
us_pacific = timezone('US/Pacific')
time = datetime.now(us_pacific)
us_time = time.strftime('%Y-%m-%d')
print('Current US/Pacific time: ', us_time)

Current US/Pacific time:  2018-09-28


In [11]:
# write the food list to a csv file
path = '/Users/7w0r4ng3s/Documents/menu_scraping/data/{}.csv'.format(us_time)
print('Current path: ', path)
path_2 = '/Users/7w0r4ng3s/Documents/menu_scraping/data/'

Current path:  /Users/7w0r4ng3s/Documents/menu_scraping/data/2018-09-28.csv


In [12]:
def write_data():
    with open(path, "w") as output:
        writer = csv.writer(output, lineterminator='\n')
        for key, val in zip(food_list, pref):
            writer.writerow([key, val])
    print('write_csv: COMPLETED')
    # add index and column names
    df = pd.read_csv(path, names=['food', 'pref'])
    df.index.names = ['index']
    df.to_csv(path)
    print('add_column_name: COMPLETED')
    
write_data()

write_csv: COMPLETED
add_column_name: COMPLETED


In [13]:
# def merge_data():
#     # TODO: Figure out a way to get rid of the unnamed: 0 column
#     # TODO: Modify merge_data() so that new data can be append to data.csv
#     files = [f for f in os.listdir('.') if os.path.isfile(f)]

#     merged = []

#     for f in files:
#         filename, ext = os.path.splitext(f)
#         if ext == '.csv':
#             read = pd.read_csv(f)
#             merged.append(read)

#     result = pd.concat(merged)
#     result.to_csv('data.csv')
    
# merge_data()

In [14]:
def append_data():
    df1 = pd.read_csv('Data.csv', index_col='index')
    df1.index.names = ['index']
    df2 = pd.read_csv(path, index_col='index')
    df3 = df1.append(df2).reset_index()
    df3 = df3.drop('index', 1).sort_values('food').reset_index().drop('index', 1)
    df3.index.names = ['index']
    df3.to_csv('Data.csv')
    
append_data()

In [16]:
FoodTools.search()

Search food: char
Do you mean:
1 Char Siu Sauce
2 Char Siu Seitan
3 Charro Beans
4 Garlic Rainbow Chard

Enter the index: 2
--------------------------------------------------------------------------------
Out of 1 time(s), you like [Char Siu Seitan] 1 time(s).
--------------------------------------------------------------------------------


1.0